# OpenMathReasoning データセットのトークン長分析

このノートブックでは、train.parquetファイルのprompt+responseを連結し、Qwen3でトークン化して最長トークン数を求めます。

In [ ]:
# 必要なライブラリのインストール
!pip install transformers torch pandas datasets pyarrow matplotlib seaborn tqdm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
import os
from tqdm import tqdm
import json

## 1. データの読み込み

In [ ]:
# データファイルのパス
data_path = os.path.expanduser("/kaggle/input/open-math-reasoning-10fold/train.parquet")

# parquetファイルの読み込み
print(f"データファイルを読み込み中: {data_path}")
df = pd.read_parquet(data_path)
print(f"データ件数: {len(df)}")
print(f"カラム: {df.columns.tolist()}")

In [ ]:
# データの構造を確認
print("最初のサンプルの構造:")
sample = df.iloc[0]
print(f"prompt型: {type(sample['prompt'])}")
print(f"extra_info型: {type(sample['extra_info'])}")

# promptの内容確認
print("\npromptの構造:")
print(sample['prompt'])

# extra_infoの内容確認
print("\nextra_infoの構造:")
if isinstance(sample['extra_info'], str):
    extra_info = json.loads(sample['extra_info'])
else:
    extra_info = sample['extra_info']
print(extra_info.keys())

## 2. Prompt + Response の抽出と連結

In [ ]:
def extract_prompt_response(row):
    """各行からpromptとresponseを抽出して連結"""
    # promptからユーザーメッセージを抽出
    prompt_data = row['prompt']
    if isinstance(prompt_data, str):
        prompt_data = json.loads(prompt_data)
    
    user_content = ""
    for msg in prompt_data:
        if msg.get('role') == 'user':
            user_content = msg.get('content', '')
            break
    
    # extra_infoからresponseを抽出
    extra_info = row['extra_info']
    if isinstance(extra_info, str):
        extra_info = json.loads(extra_info)
    
    response = extra_info.get('answer', '')
    
    # チャット形式で連結
    combined_text = f"User: {user_content}\n\nAssistant: {response}"
    
    return combined_text, user_content, response

In [ ]:
# 全データに対してprompt+response抽出を実行
print("prompt + response を抽出中...")
results = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="データ処理中"):
    combined_text, prompt, response = extract_prompt_response(row)
    results.append({
        'index': idx,
        'combined_text': combined_text,
        'prompt': prompt,
        'response': response,
        'prompt_length': len(prompt),
        'response_length': len(response),
        'combined_length': len(combined_text)
    })

# 結果をDataFrameに変換
text_df = pd.DataFrame(results)
print(f"処理完了: {len(text_df)} サンプル")

In [ ]:
# 文字数統計を表示
print("=== 文字数統計 ===")
print(f"Prompt文字数 - 平均: {text_df['prompt_length'].mean():.1f}, 最大: {text_df['prompt_length'].max()}, 最小: {text_df['prompt_length'].min()}")
print(f"Response文字数 - 平均: {text_df['response_length'].mean():.1f}, 最大: {text_df['response_length'].max()}, 最小: {text_df['response_length'].min()}")
print(f"Combined文字数 - 平均: {text_df['combined_length'].mean():.1f}, 最大: {text_df['combined_length'].max()}, 最小: {text_df['combined_length'].min()}")

# 最長文字数のサンプルを表示
max_idx = text_df['combined_length'].idxmax()
print(f"\n最長文字数サンプル (インデックス: {max_idx}, 文字数: {text_df.loc[max_idx, 'combined_length']})")
print("=" * 50)
print(text_df.loc[max_idx, 'combined_text'][:1000] + "..." if len(text_df.loc[max_idx, 'combined_text']) > 1000 else text_df.loc[max_idx, 'combined_text'])

## 3. Qwen3 トークナイザーの設定

In [ ]:
# Qwen3トークナイザーをロード
# 利用可能なQwen3系モデルを試す
model_names = [
    "Qwen/Qwen3-0.6B",
    #"Qwen/Qwen2.5-1.5B", 
    #"Qwen/Qwen2.5-3B",
    #"Qwen/Qwen2.5-7B"
]

tokenizer = None
used_model = None

for model_name in model_names:
    try:
        print(f"トークナイザーを試行中: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        used_model = model_name
        print(f"成功: {model_name} のトークナイザーを使用")
        break
    except Exception as e:
        print(f"失敗: {model_name} - {e}")
        continue

if tokenizer is None:
    raise RuntimeError("利用可能なQwen3トークナイザーが見つかりませんでした")

print(f"\n使用するトークナイザー: {used_model}")
print(f"語彙サイズ: {tokenizer.vocab_size}")

## 4. トークン化とトークン長分析

In [ ]:
def tokenize_text(text):
    """テキストをトークン化してトークン数を返す"""
    try:
        tokens = tokenizer.encode(text, add_special_tokens=True)
        return len(tokens)
    except Exception as e:
        print(f"トークン化エラー: {e}")
        return 0

In [ ]:
# 全サンプルのトークン数を計算
print("トークン化を実行中...")

# バッチ処理でメモリ効率を向上
batch_size = 100
token_counts = []

for i in tqdm(range(0, len(text_df), batch_size), desc="トークン化中"):
    batch = text_df.iloc[i:i+batch_size]
    batch_token_counts = []
    
    for _, row in batch.iterrows():
        token_count = tokenize_text(row['combined_text'])
        batch_token_counts.append(token_count)
    
    token_counts.extend(batch_token_counts)

# 結果をDataFrameに追加
text_df['token_count'] = token_counts
print(f"トークン化完了: {len(text_df)} サンプル")

In [ ]:
# トークン数統計を計算・表示
print("=== トークン数統計 ===")
print(f"平均トークン数: {text_df['token_count'].mean():.1f}")
print(f"最大トークン数: {text_df['token_count'].max()}")
print(f"最小トークン数: {text_df['token_count'].min()}")
print(f"中央値: {text_df['token_count'].median():.1f}")
print(f"標準偏差: {text_df['token_count'].std():.1f}")

# パーセンタイル情報
percentiles = [50, 75, 90, 95, 99]
print("\n=== パーセンタイル分布 ===")
for p in percentiles:
    value = np.percentile(text_df['token_count'], p)
    print(f"{p}%tile: {value:.0f} tokens")

In [ ]:
# 最長トークンのサンプルを表示
max_token_idx = text_df['token_count'].idxmax()
max_token_row = text_df.loc[max_token_idx]

print(f"\n=== 最長トークンサンプル ===")
print(f"インデックス: {max_token_idx}")
print(f"トークン数: {max_token_row['token_count']}")
print(f"文字数: {max_token_row['combined_length']}")
print(f"文字数/トークン数比: {max_token_row['combined_length']/max_token_row['token_count']:.2f}")
print("\n=== サンプル内容 ===")
print("=" * 80)
sample_text = max_token_row['combined_text']
if len(sample_text) > 2000:
    print(sample_text[:1000])
    print("\n... [中略] ...\n")
    print(sample_text[-1000:])
else:
    print(sample_text)
print("=" * 80)

## 5. 結果の可視化

In [ ]:
# トークン数分布のヒストグラム
plt.figure(figsize=(15, 10))

# 全体の分布
plt.subplot(2, 2, 1)
plt.hist(text_df['token_count'], bins=50, alpha=0.7, edgecolor='black')
plt.title('トークン数分布（全体）')
plt.xlabel('トークン数')
plt.ylabel('頻度')
plt.axvline(text_df['token_count'].mean(), color='red', linestyle='--', label=f'平均: {text_df["token_count"].mean():.0f}')
plt.axvline(text_df['token_count'].median(), color='green', linestyle='--', label=f'中央値: {text_df["token_count"].median():.0f}')
plt.legend()

# 95%ile以下に絞った分布
plt.subplot(2, 2, 2)
p95 = np.percentile(text_df['token_count'], 95)
filtered_tokens = text_df[text_df['token_count'] <= p95]['token_count']
plt.hist(filtered_tokens, bins=50, alpha=0.7, edgecolor='black')
plt.title(f'トークン数分布（95%ile以下: ≤{p95:.0f}）')
plt.xlabel('トークン数')
plt.ylabel('頻度')

# 文字数 vs トークン数の散布図
plt.subplot(2, 2, 3)
sample_idx = np.random.choice(len(text_df), min(1000, len(text_df)), replace=False)
sample_data = text_df.iloc[sample_idx]
plt.scatter(sample_data['combined_length'], sample_data['token_count'], alpha=0.6)
plt.title('文字数 vs トークン数（サンプル1000件）')
plt.xlabel('文字数')
plt.ylabel('トークン数')

# 文字数/トークン数比の分布
plt.subplot(2, 2, 4)
char_token_ratio = text_df['combined_length'] / text_df['token_count']
plt.hist(char_token_ratio, bins=50, alpha=0.7, edgecolor='black')
plt.title('文字数/トークン数比の分布')
plt.xlabel('文字数/トークン数')
plt.ylabel('頻度')
plt.axvline(char_token_ratio.mean(), color='red', linestyle='--', label=f'平均: {char_token_ratio.mean():.2f}')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 上位トークン数サンプルの一覧
print("=== 上位20サンプル（トークン数順） ===")
top_samples = text_df.nlargest(20, 'token_count')[['index', 'token_count', 'combined_length', 'prompt_length', 'response_length']]
top_samples['char_token_ratio'] = top_samples['combined_length'] / top_samples['token_count']
print(top_samples.to_string(index=False))

In [ ]:
# トークン長区間別の分布
bins = [0, 1000, 2000, 4000, 8000, 16000, float('inf')]
labels = ['~1K', '1K-2K', '2K-4K', '4K-8K', '8K-16K', '16K+']
text_df['token_range'] = pd.cut(text_df['token_count'], bins=bins, labels=labels, right=False)

print("=== トークン長区間別分布 ===")
range_counts = text_df['token_range'].value_counts().sort_index()
range_percentage = (range_counts / len(text_df) * 100).round(2)

for range_name, count in range_counts.items():
    percentage = range_percentage[range_name]
    print(f"{range_name:>8}: {count:>6}件 ({percentage:>5.1f}%)")

print(f"\n総件数: {len(text_df)}")
print(f"最大トークン数: {text_df['token_count'].max()}")

## 6. 結果のまとめ

In [ ]:
# 結果サマリーを作成
summary = {
    'dataset_info': {
        'total_samples': len(text_df),
        'data_source': 'OpenMathReasoning train.parquet',
        'tokenizer_model': used_model
    },
    'token_statistics': {
        'max_tokens': int(text_df['token_count'].max()),
        'min_tokens': int(text_df['token_count'].min()),
        'mean_tokens': float(text_df['token_count'].mean()),
        'median_tokens': float(text_df['token_count'].median()),
        'std_tokens': float(text_df['token_count'].std())
    },
    'percentiles': {
        f'{p}th': float(np.percentile(text_df['token_count'], p))
        for p in [50, 75, 90, 95, 99]
    },
    'character_statistics': {
        'max_chars': int(text_df['combined_length'].max()),
        'mean_chars': float(text_df['combined_length'].mean()),
        'char_token_ratio': float((text_df['combined_length'] / text_df['token_count']).mean())
    }
}

print("=== 分析結果サマリー ===")
print(json.dumps(summary, indent=2, ensure_ascii=False))

# 結果をファイルに保存
output_path = "/Users/kumakura/tmp/llm-bridge-sahara/ttoken_analysis_summary.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)
print(f"\n結果を保存しました: {output_path}")